In [614]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from IPython import display
import pprint
import pymongo
import random
from collections import defaultdict
import numpy as np
from bokeh.io import output_notebook
from climatools.cliradlw.cliradlw_latest import *
from climatools.html.html import *
from climatools.spectrum import *
from climatools.atm import *
output_notebook()
pd.set_option('max_colwidth', -1)

Loading BokehJS ...

# CLIRAD-LW latest  
CLIRAD vs CRD (vs WGT igg=10)

In [299]:
CLIENT = pymongo.MongoClient(host='localhost', port=27017)

In [610]:
LINKS = pd.DataFrame()
LINKS.columns.name = ['spectral band']
LINKS.index.name = ['absorber(s)']

In [659]:
LINKS.fillna('', inplace=True)
LINKS.sort_index(axis=1, inplace=True)
LINKS.sort_index(axis=0, inplace=True)
display.display(display.HTML(LINKS.to_html(escape=False)))

[spectral band],1,3,4,5,6,7,8,9,10,11
[absorber(s)],,,,,,,,,,
{'co2': 0.0004},,mls saw trp,mls saw trp,,,,,,,
"{'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}",,mls saw trp,,,,,,,,
"{'h2o': 'atmpro', 'co2': 0.0004, 'o3': 'atmpro'}",,,,,,mls saw trp,,,,
"{'h2o': 'atmpro', 'co2': 0.0004}",,,mls saw trp,mls saw trp,mls saw trp,,,,,mls saw trp
"{'h2o': 'atmpro', 'n2o': 3.2e-07, 'ch4': 1.8e-06}",,,,,,,,mls saw trp,,
{'h2o': 'atmpro'},mls saw trp,,,,,,mls saw trp,,mls saw trp,
{'o3': 'atmpro'},,,,,,mls saw trp,,,,


In [636]:
def html_case(band=None, molecule=None):
    allgs = {g: molecule[g] if g in molecule else 0 for g in ('h2o', 'co2', 'o3', 'n2o', 'ch4')}
    allgs = '_'.join(f"{g}_{conc}" for g, conc in allgs.items())
    fname = f'''band{band:02d}_-_{allgs}.ipynb'''
    furl = (f'''https://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/'''
            f'''blob/master/results_-_single_spectral_band/{fname}''')
    urls = {atmpro:furl + f'#{atmpro}' for atmpro in ATMPRO.keys()}
    htmls = [f'''<a href="{url}">{atmpro}</a>''' for atmpro, url in urls.items()]
    return ' '.join(htmls)

def update_table(df, band=None, molecule=None):
    if band == None or molecule == None: return df
    html = html_case(band=band, molecule=molecule)
    df.loc[str(molecule), band] = html

### $H_2O$ 

In [625]:
band, molecule = 1, {'h2o':'atmpro'}
html = html_case(band=band, molecule=molecule)
LINKS.loc[str(molecule), band] = html

In [632]:
band, molecule = 3, {'h2o':'atmpro', 'co2':400e-6, 'n2o':3.2e-7}
html = html_case(band=band, molecule=molecule)
LINKS.loc[str(molecule), band] = html

In [637]:
band, molecule = 4, {'h2o':'atmpro', 'co2':400e-6}
html = html_case(band=band, molecule=molecule)
LINKS.loc[str(molecule), band] = html

In [639]:
band, molecule = 5, {'h2o':'atmpro', 'co2':400e-6}
update_table(LINKS, band=band, molecule=molecule)

In [641]:
band, molecule = 6, {'h2o':'atmpro', 'co2':400e-6}
update_table(LINKS, band=band, molecule=molecule)

In [645]:
band, molecule = 7, {'h2o':'atmpro', 'co2':400e-6, 'o3':'atmpro'}
update_table(LINKS, band=band, molecule=molecule)

In [644]:
band, molecule = 8, {'h2o':'atmpro'}
update_table(LINKS, band=band, molecule=molecule)

In [648]:
band, molecule = 9, {'h2o':'atmpro', 'n2o':3.2e-7, 'ch4':1.8e-6}
update_table(LINKS, band=band, molecule=molecule)

In [649]:
band, molecule = 10, {'h2o':'atmpro'}
update_table(LINKS, band=band, molecule=molecule)

In [652]:
band, molecule = 11, {'h2o':'atmpro', 'co2':400e-6}
update_table(LINKS, band=band, molecule=molecule)

### $CO_2$

In [655]:
band, molecule = 3, {'co2':400e-6}
update_table(LINKS, band=band, molecule=molecule)

In [656]:
band, molecule = 4, {'co2':400e-6}
update_table(LINKS, band=band, molecule=molecule)

### $O_3$

In [658]:
band, molecule = 7, {'o3':'atmpro'}
update_table(LINKS, band=band, molecule=molecule)

# Fin

In [650]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')
